In [124]:
import pandas as pd
from json import loads, dumps


In [125]:
summary96 = pd.read_csv("../processed-tables/merge96.csv")
summary21 = pd.read_csv("../processed-tables/merge21.csv")

census96 = pd.read_csv("../input-files/Census1996.csv")
census21 = pd.read_csv("../input-files/Census2021.csv")

station_distance = pd.read_csv("../input-files/transit-station-distances.csv") #this file is a table of distance of each station from their transit line origin. 
cdData = pd.read_csv("../input-files/CD-data.csv")

In [126]:
#pivot! sum the intersected areas, some transit buffers have several rows of one DA due to the intersecting cutting a DA up into multiple smaller pieces within the buffer, 
sum96_pivot = pd.pivot_table(summary96, values = ['Int_Area'], index = ['NAME', 'LOCATION_N', 'ORIG_SEQ', 'TECHNOLOGY', 'EAUID','DA_Area'], aggfunc = "sum").reset_index()
sum21_pivot = pd.pivot_table(summary21, values = ['Int_Area'], index = ['NAME', 'LOCATION_N','ORIG_SEQ', 'TECHNOLOGY', 'DAUID','DA_Area'], aggfunc = "sum").reset_index()

In [127]:

#calculating the ration of each DA within the transit buffer. 
sum96_pivot['Area Ratio'] = sum96_pivot['Int_Area']/sum96_pivot['DA_Area']
sum21_pivot['Area Ratio'] = sum21_pivot['Int_Area']/sum21_pivot['DA_Area']

In [128]:
#join the census data with the summary of area layer. 
sum96 = pd.merge(sum96_pivot, census96, on = "EAUID")
sum21 = pd.merge(sum21_pivot, census21, on = "DAUID")

#get the list of census variables to calculate the portions of values within the buffer.
#list of values that are averages and does not need spatial interpolation
averageList = ["Index", "Int_Area", 'Average Total Individual Income', 'Average Total Household Income', 'Average number of rooms per dwelling']

censusList96= sum96.columns.tolist()[-15:]
censusList21= sum21.columns.tolist()[-15:]

#create an index for summing
sum96['ORIG_SEQ'] = sum96['ORIG_SEQ'].astype('str')
sum21['ORIG_SEQ'] = sum21['ORIG_SEQ'].astype('str')

sum96['Index'] = sum96['NAME'] + "_" + sum96['LOCATION_N'] + "_" + sum96['ORIG_SEQ']
sum21['Index'] = sum21['NAME'] + "_" + sum21['LOCATION_N'] + "_" + sum21['ORIG_SEQ']

Calculating the Variables that need to be summed

In [129]:
#remove the average values from the interpolation
i=2
while i < len(averageList): 
    censusList96.remove(averageList[i])
    censusList21.remove(averageList[i])
    i+=1

In [130]:
#loop through the census lists to calculate the ratio wtihin the buffer. 
intList96 = []
intList21 = []

i=0
while i < len(censusList96):
    sum96[f'{censusList96[i]} 96'] = sum96[f'{censusList96[i]}']*sum96['Area Ratio']
    sum21[f'{censusList21[i]} 21'] = sum21[f'{censusList21[i]}']*sum21['Area Ratio']
    intList96.append(f'{censusList96[i]} 96')
    intList21.append(f'{censusList96[i]} 21')
    i+=1

In [131]:
#pivot table of the intesected census variables that requires summing
summed_sum96 = pd.pivot_table(sum96, values= intList96, index = ['Index'], aggfunc='sum', sort = False).reset_index()
summed_sum96['Owner Renter Ratio 96'] = summed_sum96['Owner 96']/summed_sum96['Renter 96'] #ratio of owner vs renter
summed_sum21 = pd.pivot_table(sum21, values= intList21, index = ['Index'], aggfunc='sum', sort = False).reset_index()
summed_sum21['Owner Renter Ratio 21'] = summed_sum21['Owner 21']/summed_sum21['Renter 21']


In [132]:

summed_sum21[['Transit Line', 'Station Name','Point ID']] = summed_sum21['Index'].str.split("_", expand = True)
summed_sum96[['Transit Line', 'Station Name','Point ID']] = summed_sum96['Index'].str.split("_", expand = True)

summed_sum21['Point ID'] = summed_sum21['Point ID'].astype("int")
summed_sum96['Point ID'] = summed_sum96['Point ID'].astype("int")
summed_sum21.sort_values(['Transit Line', 'Point ID']).head(30)


,Index,Population 21,Total Occupied Dwellings 21,Single-detached house 21,Semi-detached house 21,Row house 21,Apartment or flat 21,Apartment fewer than five storeys 21,Apartment five or more storeys 21,Other single-attached house 21,Movable dwelling 21,Owner 21,Renter 21,Owner Renter Ratio 21,Transit Line,Station Name,Point ID
49,Eglinton Crosstown LRT_Kennedy_1,15147.070620,5639.208437,998.480910,229.682813,253.979625,370.227969,406.621926,3319.870730,78.103996,0.0,2263.684550,3393.867763,0.666993,Eglinton Crosstown LRT,Kennedy,1
47,Eglinton Crosstown LRT_Ionview_2,15677.667939,5884.010169,880.353622,408.855203,219.926897,342.535852,371.285757,3539.646539,132.822033,0.0,2117.858746,3765.285954,0.562470,Eglinton Crosstown LRT,Ionview,2
48,Eglinton Crosstown LRT_Ionview_3,12481.023079,4617.969697,681.893672,407.936671,232.096420,286.454460,281.245962,2633.509867,95.000575,0.0,1616.390309,2961.573843,0.545788,Eglinton Crosstown LRT,Ionview,3
43,Eglinton Crosstown LRT_Birchmount_4,9812.821290,3633.148050,407.730634,387.058880,285.296864,163.132202,186.477087,2107.018954,95.000575,0.0,1269.071226,2319.054697,0.547236,Eglinton Crosstown LRT,Birchmount,4
44,Eglinton Crosstown LRT_Golden Mile_5,4959.846704,1842.859586,241.392844,190.986800,234.292129,65.420142,142.708182,898.136134,77.421064,0.0,819.124113,1018.269548,0.804428,Eglinton Crosstown LRT,Golden Mile,5
45,Eglinton Crosstown LRT_Golden Mile_6,2046.900396,720.898179,272.617368,26.277215,167.048010,55.784393,100.615040,104.798660,0.000000,0.0,601.469745,129.085631,4.659463,Eglinton Crosstown LRT,Golden Mile,6
46,Eglinton Crosstown LRT_Hakimi Lebovic_7,2858.115352,1131.961872,370.901140,9.382003,56.932345,64.592655,486.630983,147.647093,0.000000,0.0,566.021948,566.517939,0.999124,Eglinton Crosstown LRT,Hakimi Lebovic,7
17,Eglinton Crosstown LRT_Pharmacy_8,7585.154443,3390.985582,549.431851,28.503210,20.625897,168.017095,1320.753787,1313.152197,0.000000,0.0,1334.543044,2054.833332,0.649465,Eglinton Crosstown LRT,Pharmacy,8
16,Eglinton Crosstown LRT_O'Connor_9,9869.105028,4374.605319,540.189543,53.074864,122.169094,151.542770,1715.476475,1803.201931,0.000000,0.0,1730.420446,2652.303870,0.652422,Eglinton Crosstown LRT,O'Connor,9
1,Eglinton Crosstown LRT_O'Connor_10,10020.855751,4536.452536,557.122130,55.008979,134.174589,135.304701,2124.588935,1539.320569,0.000000,0.0,1674.566517,2866.982830,0.584087,Eglinton Crosstown LRT,O'Connor,10


Calculating the Varaibles that are only averages

In [133]:
averageTable96 = sum96[averageList]
averageTable21 = sum21[averageList]

In [134]:
#calculate the total area of each buffer 
Buffer_Area_96 = pd.pivot_table(sum96, values = ['Int_Area'], index = ['Index'], aggfunc = "sum").reset_index()
Buffer_Area_21 = pd.pivot_table(sum21, values = ['Int_Area'], index = ['Index'], aggfunc = "sum").reset_index()

Buffer_Area_96 = Buffer_Area_96.rename(columns = {"Int_Area" : "Buffer Area"})
Buffer_Area_21 = Buffer_Area_21.rename(columns = {"Int_Area" : "Buffer Area"})


In [135]:
Buffer_Area_21

,Index,Buffer Area
0,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,2.011076e+06
1,Eglinton Crosstown LRT_Aga Khan Park & Museum_16,2.011098e+06
2,Eglinton Crosstown LRT_Avenue_31,2.010988e+06
3,Eglinton Crosstown LRT_Avenue_32,2.010957e+06
4,Eglinton Crosstown LRT_Birchmount_4,2.010966e+06
...,...,...
414,Yonge North Subway Extension_Royal Orchard_15,2.010535e+06
415,Yonge North Subway Extension_Royal Orchard_16,2.010974e+06
416,Yonge North Subway Extension_Steeles_5,2.010950e+06
417,Yonge North Subway Extension_Steeles_6,2.010874e+06


In [136]:
#join back to the sum table
averageTable96 = pd.merge(averageTable96, Buffer_Area_96, on = "Index")
averageTable21 = pd.merge(averageTable21, Buffer_Area_21, on = "Index")

In [137]:
averageTable21

,Index,Int_Area,Average Total Individual Income,Average Total Household Income,Average number of rooms per dwelling,Buffer Area
0,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,3898.201139,49600.0,80400.0,4.6,2.011076e+06
1,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,79735.154554,65500.0,104500.0,4.1,2.011076e+06
2,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,105558.849642,48600.0,129500.0,5.3,2.011076e+06
3,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,29800.696978,32200.0,48800.0,3.3,2.011076e+06
4,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,62261.690114,34000.0,61400.0,2.9,2.011076e+06
...,...,...,...,...,...,...
10669,Yonge North Subway Extension_Steeles_6,5769.545178,46800.0,120000.0,7.3,2.010874e+06
10670,Yonge North Subway Extension_Steeles_6,33908.939214,50000.0,128000.0,7.9,2.010874e+06
10671,Yonge North Subway Extension_Steeles_6,2872.142659,45800.0,98000.0,7.1,2.010874e+06
10672,Yonge North Subway Extension_Steeles_6,5795.094272,41400.0,91000.0,6.1,2.010874e+06


In [138]:
weightedVales96 = []
weightedVales21 = []
averageTable96 = averageTable96.fillna(0)
averageTable21 = averageTable21.fillna(0)
averageTable96['Area Ratio'] = averageTable96['Int_Area']/averageTable96['Buffer Area']
averageTable21['Area Ratio'] = averageTable21['Int_Area']/averageTable21['Buffer Area']
i = 2
while i < len(averageList):
    averageTable96[f'Weighted {averageList[i]} 96'] = averageTable96[f'{averageList[i]}']*(averageTable96['Area Ratio'])
    averageTable21[f'Weighted {averageList[i]} 21'] = averageTable21[f'{averageList[i]}']*(averageTable21['Area Ratio'])
    weightedVales96.append(f'Weighted {averageList[i]} 96')
    weightedVales21.append(f'Weighted {averageList[i]} 21')

    i+=1

In [139]:
averageTable96

,Index,Int_Area,Average Total Individual Income,Average Total Household Income,Average number of rooms per dwelling,Buffer Area,Area Ratio,Weighted Average Total Individual Income 96,Weighted Average Total Household Income 96,Weighted Average number of rooms per dwelling 96
0,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,123203.555593,21162.0,44619.0,5.7,1.886323e+06,0.065314,1382.177724,2914.251388,0.372291
1,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,3172.074198,20549.0,45901.0,4.9,1.886323e+06,0.001682,34.555563,77.187935,0.008240
2,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,1617.051195,19599.0,43138.0,4.8,1.886323e+06,0.000857,16.801251,36.980069,0.004115
3,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,215746.460426,16796.0,30868.0,4.6,1.886323e+06,0.114374,1921.027063,3530.499129,0.526121
4,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,54208.636466,12178.0,21435.0,3.7,1.886323e+06,0.028738,349.968045,615.993187,0.106330
...,...,...,...,...,...,...,...,...,...,...
10518,Yonge North Subway Extension_Clark_9,287015.005011,34827.0,74250.0,6.3,2.010941e+06,0.142727,4970.743197,10597.458361,0.899178
10519,Yonge North Subway Extension_Clark_9,102893.906038,39393.0,91656.0,8.3,2.010941e+06,0.051167,2015.623295,4689.766424,0.424686
10520,Yonge North Subway Extension_Clark_9,18118.794178,47343.0,81705.0,6.2,2.010941e+06,0.009010,426.565498,736.170796,0.055863
10521,Yonge North Subway Extension_Clark_9,138725.526164,68010.0,148540.0,8.2,2.010941e+06,0.068985,4691.695463,10247.087840,0.565680


In [140]:
#pivot table of the intesected census variables that requires summing
weightedsummed_sum96 = pd.pivot_table(averageTable96, values= weightedVales96, index = ['Index'], aggfunc='sum', sort = False).reset_index()
weightedsummed_sum21 = pd.pivot_table(averageTable21, values= weightedVales21, index = ['Index'], aggfunc='sum', sort = False).reset_index()

In [141]:
weightedsummed_sum21

,Index,Weighted Average Total Individual Income 21,Weighted Average Total Household Income 21,Weighted Average number of rooms per dwelling 21
0,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,53307.719751,104112.989082,4.247876
1,Eglinton Crosstown LRT_O'Connor_10,46890.218832,102585.095372,4.790477
2,Eglinton Crosstown LRT_Sloane_11,50278.239048,112439.834298,5.114857
3,Eglinton Crosstown LRT_Sloane_12,50551.271792,113748.130325,5.262587
4,Eglinton Crosstown LRT_Wynford_13,50726.347961,107968.869584,4.822548
...,...,...,...,...
414,Yonge North Subway Extension_Clark_11,102839.650464,224743.405591,6.972108
415,Yonge North Subway Extension_Clark_8,54806.309094,123269.671315,5.619548
416,Yonge North Subway Extension_Clark_9,65230.398632,143058.337498,5.880066
417,Yonge North Subway Extension_Steeles_7,51351.009213,117082.136599,5.548577


In [142]:
summed_sum96 = pd.merge(summed_sum96, weightedsummed_sum96,  on = 'Index')
summed_sum21 = pd.merge(summed_sum21, weightedsummed_sum21,  on = 'Index')

In [143]:
summed_sum96.insert(0, 'Transit Line', summed_sum96.pop('Transit Line'))
summed_sum96.insert(0, 'Point ID', summed_sum96.pop('Point ID'))
summed_sum96.insert(0, 'Station Name', summed_sum96.pop('Station Name'))

summed_sum21.insert(0, 'Transit Line', summed_sum21.pop('Transit Line'))
summed_sum21.insert(0, 'Point ID', summed_sum21.pop('Point ID'))
summed_sum21.insert(0, 'Station Name', summed_sum21.pop('Station Name'))

In [144]:
summed_sum21 = summed_sum21.drop(columns = ['Station Name', 'Point ID', 'Transit Line'], axis = 1)
summary = pd.merge(summed_sum96, summed_sum21, on = "Index")
#summary = summary.drop(columns = 'Index', axis =1).sort_values(['Transit Line', 'Point ID'])

#adjust for inflation
summary['Weighted Average Total Individual Income 96'] = summary['Weighted Average Total Individual Income 96']*1.6275626
summary['Weighted Average Total Household Income 96'] = summary ['Weighted Average Total Household Income 96'] *1.6275626

Getting the xy coordinates of the points and the stations to identify the closest point a station is. 

In [145]:
#pivot! sum the intersected areas, some transit buffers have several rows of one DA due to the intersecting cutting a DA up into multiple smaller pieces within the buffer, 
pointsPivot = pd.pivot_table(summary96, values = "Int_Area", index = ['NAME', 'LOCATION_N', 'ORIG_SEQ','PointX','PointY']).reset_index()
pointsPivot['ORIG_SEQ'] = pointsPivot['ORIG_SEQ'].astype('str')
pointsPivot['Index'] = pointsPivot['NAME'] + "_" + pointsPivot['LOCATION_N'] + "_" + pointsPivot['ORIG_SEQ']

stationTable = pd.read_csv("../processed-tables/station.csv")



In [146]:
pointsPivot['ORIG_SEQ'] = pointsPivot['ORIG_SEQ'].astype('int')
pointsPivot.sort_values(["NAME", "ORIG_SEQ"])

,NAME,LOCATION_N,ORIG_SEQ,PointX,PointY,Int_Area,Index
24,Eglinton Crosstown LRT,Kennedy,1,639765.013955,4.843599e+06,31919.970344,Eglinton Crosstown LRT_Kennedy_1
20,Eglinton Crosstown LRT,Ionview,2,639371.449180,4.843547e+06,25137.866472,Eglinton Crosstown LRT_Ionview_2
21,Eglinton Crosstown LRT,Ionview,3,638992.773426,4.843419e+06,25460.410331,Eglinton Crosstown LRT_Ionview_3
4,Eglinton Crosstown LRT,Birchmount,4,638610.440283,4.843306e+06,26116.626773,Eglinton Crosstown LRT_Birchmount_4
17,Eglinton Crosstown LRT,Golden Mile,5,638230.511185,4.843181e+06,35281.267382,Eglinton Crosstown LRT_Golden Mile_5
...,...,...,...,...,...,...,...
398,Yonge North Subway Extension,Bridge,17,627264.285563,4.854435e+06,36569.309750,Yonge North Subway Extension_Bridge_17
399,Yonge North Subway Extension,Bridge,18,626993.378389,4.854718e+06,30464.192353,Yonge North Subway Extension_Bridge_18
400,Yonge North Subway Extension,Bridge,19,626765.798593,4.855043e+06,37240.614181,Yonge North Subway Extension_Bridge_19
409,Yonge North Subway Extension,High Tech,20,626561.685020,4.855387e+06,52922.694399,Yonge North Subway Extension_High Tech_20


In [147]:
pointsPivot.values.tolist()[1]

['Eglinton Crosstown LRT',
 'Aga Khan Park & Museum',
 16,
 634033.1536487162,
 4842274.652622435,
 25783.29804206577,
 'Eglinton Crosstown LRT_Aga Khan Park & Museum_16']

In [148]:

transit_lines = stationTable['NAME'].unique()
header = stationTable.columns.tolist()
stationvalues = stationTable.values.tolist()
points = pointsPivot.values.tolist()

In [149]:
closest = []
for transit in transit_lines:
    
    for station in stationvalues:
        if station[4] == transit:
            x = station[5]
            y = station[6]
            distances = []
            for point in points:
                if point[0] == transit and point[1] == station[1]:
                    ptX = point[3]
                    ptY = point[4]
                    distance = ((ptX - x)**2 + (ptY - y)**2)**1/2 #calculate distances
                    if len(distances) == 0:
                        distances.append(point[6]) #point[6] is the index, add to distances list first
                        distances.append(distance)
                        distances.append("Label")
                    elif distance < distances[1]: #if the new distance is shorter than the existing
                        distances[0] = point[6]
                        distances[1] = distance
                    else:
                        continue
                else:
                    continue
            closest.append(distances)
        else: 
            continue

print(closest)
labelTable = pd.DataFrame.from_records(closest, columns = ["Index", "Distance", "Label"])


[['Eglinton Crosstown LRT_Avenue_31', 10183.549365350473, 'Label'], ['Eglinton Crosstown LRT_Forest Hill_35', 342.0980993027872, 'Label'], ['Eglinton Crosstown LRT_Leaside_25', 3500.960718140341, 'Label'], ['Eglinton Crosstown LRT_Sloane_11', 4151.983065250472, 'Label'], ['Eglinton Crosstown LRT_Birchmount_4', 3713.3646198855236, 'Label'], ['Eglinton Crosstown LRT_Caledonia_43', 2835.2333005144205, 'Label'], ['Eglinton Crosstown LRT_Chaplin_33', 3757.0295795973125, 'Label'], ['Eglinton Crosstown LRT_Science Centre_17', 10691.761437603225, 'Label'], ['Eglinton Crosstown LRT_Fairbank_40', 1302.4509037292726, 'Label'], ['Eglinton Crosstown LRT_Eglinton_29', 6664.399948567165, 'Label'], ['Eglinton Crosstown LRT_Cedarvale_37', 1196.0070332855018, 'Label'], ['Eglinton Crosstown LRT_Aga Khan Park & Museum_16', 15738.56908222927, 'Label'], ['Eglinton Crosstown LRT_Ionview_2', 11326.13347030142, 'Label'], ['Eglinton Crosstown LRT_Keelesdale_45', 10868.05502716589, 'Label'], ['Eglinton Crosstown

In [150]:
labelTable

,Index,Distance,Label
0,Eglinton Crosstown LRT_Avenue_31,10183.549365,Label
1,Eglinton Crosstown LRT_Forest Hill_35,342.098099,Label
2,Eglinton Crosstown LRT_Leaside_25,3500.960718,Label
3,Eglinton Crosstown LRT_Sloane_11,4151.983065,Label
4,Eglinton Crosstown LRT_Birchmount_4,3713.364620,Label
...,...,...,...
173,Ontario Line_Riverside/Leslieville_27,10627.294005,Label
174,Ontario Line_Gerrard_30,13432.525881,Label
175,Ontario Line_Cosburn_3,83.029382,Label
176,Ontario Line_Thorncliffe Park_33,33.670400,Label


In [151]:
summary = summary.merge(labelTable, how = "left", on="Index")

In [152]:
summary

,Station Name,Point ID,Transit Line,Index,Population 96,Total Occupied Dwellings 96,Single-detached house 96,Semi-detached house 96,Row house 96,Apartment or flat 96,...,Other single-attached house 21,Movable dwelling 21,Owner 21,Renter 21,Owner Renter Ratio 21,Weighted Average Total Individual Income 21,Weighted Average Total Household Income 21,Weighted Average number of rooms per dwelling 21,Distance,Label
0,Aga Khan Park & Museum,15,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,15424.687230,6468.139022,19.083456,22.526914,298.765034,22.740438,...,0.0,0.0,2757.687107,3645.293703,0.756506,53307.719751,104112.989082,4.247876,NaN,NaN
1,Aga Khan Park & Museum,16,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Aga Khan Park & Museum_16,8605.007632,2999.949614,10.674500,37.140340,283.850197,17.570209,...,0.0,0.0,1826.802308,3272.749333,0.558186,52644.515313,100267.939871,4.080553,15738.569082,Label
2,O'Connor,9,Eglinton Crosstown LRT,Eglinton Crosstown LRT_O'Connor_9,8227.204744,3556.668712,650.608524,30.679430,35.409669,62.573312,...,0.0,0.0,1730.420446,2652.303870,0.652422,45540.401807,94187.229091,4.874986,11.543770,Label
3,O'Connor,10,Eglinton Crosstown LRT,Eglinton Crosstown LRT_O'Connor_10,8712.015433,3930.738309,660.258384,44.431263,20.545696,36.909500,...,0.0,0.0,1674.566517,2866.982830,0.584087,46890.218832,102585.095372,4.790477,NaN,NaN
4,Pharmacy,8,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Pharmacy_8,6833.889805,3059.960413,690.864196,8.812575,6.704492,82.235205,...,0.0,0.0,1334.543044,2054.833332,0.649465,46582.592432,91503.825817,5.273916,516.449855,Label
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,Clark,11,Yonge North Subway Extension,Yonge North Subway Extension_Clark_11,6209.034983,2497.045838,708.997582,2.291230,365.474590,34.760842,...,0.0,0.0,1693.575263,896.332051,1.889451,102839.650464,224743.405591,6.972108,NaN,NaN
415,Clark,8,Yonge North Subway Extension,Yonge North Subway Extension_Clark_8,8095.950538,2866.321726,1060.170815,82.381722,198.331929,47.244320,...,0.0,0.0,2474.247859,1719.171042,1.439210,54806.309094,123269.671315,5.619548,NaN,NaN
416,Steeles,6,Yonge North Subway Extension,Yonge North Subway Extension_Steeles_6,7405.268404,2789.916349,1194.948414,119.950850,15.376088,117.517089,...,0.0,0.0,1919.131251,1442.422896,1.330491,50675.036890,118585.505044,5.813366,1893.492833,Label
417,Steeles,7,Yonge North Subway Extension,Yonge North Subway Extension_Steeles_7,7952.118514,2915.267604,1042.344021,122.176326,67.191564,103.914588,...,0.0,0.0,2379.813872,1520.651595,1.564996,51351.009213,117082.136599,5.548577,NaN,NaN


Re-Order some of the points so that they are showing west-east instead of east-west

In [153]:
#transit-station-reorder contains a manually rearranged order of the transit points such that the IDs are: 
#1. smallest in the west and largest in the east 2. Ontario Line's ID is messed up, 
#unless metrolinx fix it in their future files, continue to check if this table needs to be updated
transit_reorder = pd.read_csv("../processed-tables/transit-station-reorder.csv")
summary = pd.merge(summary, transit_reorder, on = "Index")

In [154]:
summary

,Station Name,Point ID,Transit Line,Index,Population 96,Total Occupied Dwellings 96,Single-detached house 96,Semi-detached house 96,Row house 96,Apartment or flat 96,...,Movable dwelling 21,Owner 21,Renter 21,Owner Renter Ratio 21,Weighted Average Total Individual Income 21,Weighted Average Total Household Income 21,Weighted Average number of rooms per dwelling 21,Distance,Label,New ID
0,Aga Khan Park & Museum,15,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Aga Khan Park & Museum_15,15424.687230,6468.139022,19.083456,22.526914,298.765034,22.740438,...,0.0,2757.687107,3645.293703,0.756506,53307.719751,104112.989082,4.247876,NaN,NaN,34
1,Aga Khan Park & Museum,16,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Aga Khan Park & Museum_16,8605.007632,2999.949614,10.674500,37.140340,283.850197,17.570209,...,0.0,1826.802308,3272.749333,0.558186,52644.515313,100267.939871,4.080553,15738.569082,Label,33
2,O'Connor,9,Eglinton Crosstown LRT,Eglinton Crosstown LRT_O'Connor_9,8227.204744,3556.668712,650.608524,30.679430,35.409669,62.573312,...,0.0,1730.420446,2652.303870,0.652422,45540.401807,94187.229091,4.874986,11.543770,Label,40
3,O'Connor,10,Eglinton Crosstown LRT,Eglinton Crosstown LRT_O'Connor_10,8712.015433,3930.738309,660.258384,44.431263,20.545696,36.909500,...,0.0,1674.566517,2866.982830,0.584087,46890.218832,102585.095372,4.790477,NaN,NaN,39
4,Pharmacy,8,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Pharmacy_8,6833.889805,3059.960413,690.864196,8.812575,6.704492,82.235205,...,0.0,1334.543044,2054.833332,0.649465,46582.592432,91503.825817,5.273916,516.449855,Label,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,Clark,11,Yonge North Subway Extension,Yonge North Subway Extension_Clark_11,6209.034983,2497.045838,708.997582,2.291230,365.474590,34.760842,...,0.0,1693.575263,896.332051,1.889451,102839.650464,224743.405591,6.972108,NaN,NaN,11
409,Clark,8,Yonge North Subway Extension,Yonge North Subway Extension_Clark_8,8095.950538,2866.321726,1060.170815,82.381722,198.331929,47.244320,...,0.0,2474.247859,1719.171042,1.439210,54806.309094,123269.671315,5.619548,NaN,NaN,8
410,Steeles,6,Yonge North Subway Extension,Yonge North Subway Extension_Steeles_6,7405.268404,2789.916349,1194.948414,119.950850,15.376088,117.517089,...,0.0,1919.131251,1442.422896,1.330491,50675.036890,118585.505044,5.813366,1893.492833,Label,6
411,Steeles,7,Yonge North Subway Extension,Yonge North Subway Extension_Steeles_7,7952.118514,2915.267604,1042.344021,122.176326,67.191564,103.914588,...,0.0,2379.813872,1520.651595,1.564996,51351.009213,117082.136599,5.548577,NaN,NaN,7


In [155]:
summary = summary.drop("Point ID", axis =1)
summary = summary.sort_values(["Transit Line", "New ID"])
summary.insert(1, 'Point ID', summary.pop('New ID')) #shift "New ID" to a new location and rename as Point ID


In [156]:
header = summary.columns.tolist()
header.insert(1, "Count")
#add the number of instances that the station has appeared in dataset, helps with labelling names in the chart
sumList = summary.values.tolist()
i = 0
a = 1
while i < len(sumList):
    if i == 0:
        sumList[i].insert(1, a)
        print(sumList[i])
        i+=1
        continue
    if sumList[i][0] == sumList[i-1][0]:
        sumList[i].insert(1, a+1)
        a+=1
    elif sumList[i][0] != sumList[i-1][0]:
        a=1
        sumList[i].insert(1, a)
    print(sumList[i])
    i+=1

for row in sumList:
    match row[0]:
        case "Vaughan Metropolitan Centre":
            row[0] = "VMC"
        case "Scarborough Town Centre":
            row[0] = "STC"
        case "North York Centre":
            row[0] = "NYC"
        case "Bellamy/Eglinton GO":
            row[0] = "Eglinton GO"
        case "Celeste/ Guildwood GO":
            row[0] = "Guildwood GO"
        case "Pan Am Sports Centre":
            row[0] = "Pan Am"
        case "Aga Khan Park & Museum":
            row[0] = "Aga Khan"

['Mount Dennis', 1, 1, 'Eglinton Crosstown LRT', 'Eglinton Crosstown LRT_Mount Dennis_48', 8276.849117351529, 3116.7477164078164, 1000.2501141985613, 146.69229911275582, 72.45640897054224, 119.97677407625119, 1037.6965005919076, 682.33745847427, 74.70970794968807, 0.0, 1212.121765821147, 1900.2893347645668, 0.6378616896101881, 31485.55811810706, 56633.52578525993, 4.432428920225137, 7849.095587216269, 3144.1263606269404, 842.6465312352586, 193.48179708004713, 108.3012277018149, 150.37158408673417, 1232.9496798710936, 588.9353076622791, 0.0, 0.0, 1243.7038212610769, 1861.4046708981216, 0.6681533793836425, 39312.27820104649, 75263.26961775031, 4.5021807499706386, 7.320783639376592e-05, 'Label']
['Mount Dennis', 2, 2, 'Eglinton Crosstown LRT', 'Eglinton Crosstown LRT_Mount Dennis_47', 7156.15498114371, 2772.83265598135, 913.3642329270309, 120.36578821656951, 48.14734934156462, 89.57588023561033, 1355.023619762792, 202.3767045578076, 56.10386238749967, 0.0, 979.1766969840078, 1795.78948142

In [157]:
"""newList = sumList
newHeader = header
/*
newHeader.insert(0, "Station ID")
#number the stations
i = 0
a = 1
print(newHeader)
while i < len(newList):
    if i == 0:
        newList[i].insert(0, a)
        print("Scenario 0" , newList[i][1])
        print(newList[i])
        i+=1
        continue
    if newList[i][0] == newList[i-1][1] and newList[i][3] == newList[i-1][4]:
        newList[i].insert(0, a)
        print("scenario 1" , newList[i][1])
        
    elif newList[i][0] != newList[i-1][1] and newList[i][3] == newList[i-1][4]:
        a+=1
        newList[i].insert(0, a)
        print("scenario 2" , newList[i][1])
        
    elif newList[i][0] != newList[i-1][1] and newList[i][3] != newList[i-1][4]:
        a=1
        newList[i].insert(0, a)
        print("scenario 3" , newList[i][1])
        a+=1
    print(newList[i])
    i+=1"""

'newList = sumList\nnewHeader = header\n/*\nnewHeader.insert(0, "Station ID")\n#number the stations\ni = 0\na = 1\nprint(newHeader)\nwhile i < len(newList):\n    if i == 0:\n        newList[i].insert(0, a)\n        print("Scenario 0" , newList[i][1])\n        print(newList[i])\n        i+=1\n        continue\n    if newList[i][0] == newList[i-1][1] and newList[i][3] == newList[i-1][4]:\n        newList[i].insert(0, a)\n        print("scenario 1" , newList[i][1])\n        \n    elif newList[i][0] != newList[i-1][1] and newList[i][3] == newList[i-1][4]:\n        a+=1\n        newList[i].insert(0, a)\n        print("scenario 2" , newList[i][1])\n        \n    elif newList[i][0] != newList[i-1][1] and newList[i][3] != newList[i-1][4]:\n        a=1\n        newList[i].insert(0, a)\n        print("scenario 3" , newList[i][1])\n        a+=1\n    print(newList[i])\n    i+=1'

In [158]:
summary = pd.DataFrame.from_records(sumList, columns = header)
summary['Transit Line'].unique().tolist()
summary

,Station Name,Count,Point ID,Transit Line,Index,Population 96,Total Occupied Dwellings 96,Single-detached house 96,Semi-detached house 96,Row house 96,...,Other single-attached house 21,Movable dwelling 21,Owner 21,Renter 21,Owner Renter Ratio 21,Weighted Average Total Individual Income 21,Weighted Average Total Household Income 21,Weighted Average number of rooms per dwelling 21,Distance,Label
0,Mount Dennis,1,1,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Mount Dennis_48,8276.849117,3116.747716,1000.250114,146.692299,72.456409,...,0.0,0.0,1243.703821,1861.404671,0.668153,39312.278201,75263.269618,4.502181,0.000073,Label
1,Mount Dennis,2,2,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Mount Dennis_47,7156.154981,2772.832656,913.364233,120.365788,48.147349,...,0.0,0.0,1127.434824,1939.443286,0.581319,40239.223643,79245.929602,4.546916,NaN,NaN
2,Keelesdale,1,3,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Keelesdale_46,8869.796244,3336.637920,1349.537818,129.338218,18.364190,...,0.0,0.0,1541.642428,1854.731312,0.831194,41497.388623,86258.875616,4.881289,NaN,NaN
3,Keelesdale,2,4,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Keelesdale_45,11578.881757,4226.396434,1689.638613,189.324730,17.707301,...,0.0,0.0,2214.372818,2142.629264,1.033484,41421.670124,88383.108605,5.287462,10868.055027,Label
4,Caledonia,1,5,Eglinton Crosstown LRT,Eglinton Crosstown LRT_Caledonia_44,12347.617570,4305.627383,1973.380380,243.208511,19.999911,...,0.0,0.0,2620.869853,2024.590756,1.294518,40365.110147,87406.824037,5.525506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,Bridge,1,17,Yonge North Subway Extension,Yonge North Subway Extension_Bridge_17,3434.076449,1135.709206,790.550964,13.354244,251.987101,...,0.0,0.0,953.984358,244.906919,3.895294,60690.864493,135058.219105,7.057803,NaN,NaN
409,Bridge,2,18,Yonge North Subway Extension,Yonge North Subway Extension_Bridge_18,2377.997171,765.488831,536.865992,11.696757,183.160034,...,0.0,0.0,979.180724,320.367884,3.056426,61161.394597,133919.461837,6.510054,NaN,NaN
410,Bridge,3,19,Yonge North Subway Extension,Yonge North Subway Extension_Bridge_19,1882.934844,579.125748,430.092857,8.632981,119.397111,...,0.0,0.0,1292.938486,554.755258,2.330647,58153.641098,124589.808683,5.893493,1405.835560,Label
411,High Tech,1,20,Yonge North Subway Extension,Yonge North Subway Extension_High Tech_20,1604.055276,462.111818,384.714878,3.377910,63.719499,...,0.0,0.0,2232.391658,975.788794,2.287782,55705.563681,115851.106723,5.452714,NaN,NaN


In [159]:
summary['Station Name'].tolist()

['Mount Dennis',
 'Mount Dennis',
 'Keelesdale',
 'Keelesdale',
 'Caledonia',
 'Caledonia',
 'Caledonia',
 'Fairbank',
 'Fairbank',
 'Oakwood',
 'Oakwood',
 'Cedarvale',
 'Forest Hill',
 'Forest Hill',
 'Chaplin',
 'Chaplin',
 'Avenue',
 'Avenue',
 'Eglinton',
 'Eglinton',
 'Mount Pleasant',
 'Mount Pleasant',
 'Leaside',
 'Leaside',
 'Leaside',
 'Laird',
 'Laird',
 'Laird',
 'Sunnybrook Park',
 'Sunnybrook Park',
 'Sunnybrook Park',
 'Science Centre',
 'Aga Khan',
 'Aga Khan',
 'Wynford',
 'Wynford',
 'Sloane',
 'Sloane',
 "O'Connor",
 "O'Connor",
 'Pharmacy',
 'Hakimi Lebovic',
 'Golden Mile',
 'Golden Mile',
 'Birchmount',
 'Ionview',
 'Ionview',
 'Kennedy',
 'Renforth',
 'Renforth',
 'Renforth',
 'Renforth',
 'Renforth',
 'Martin Grove',
 'Martin Grove',
 'Martin Grove',
 'Martin Grove',
 'Martin Grove',
 'Kipling',
 'Kipling',
 'Islington',
 'Islington',
 'Islington',
 'Royal York',
 'Royal York',
 'Royal York',
 'Scarlett',
 'Scarlett',
 'Scarlett',
 'Jane',
 'Jane',
 'Mount Denn

In [160]:
station_distance
station_distance.to_json("../../data/station-distance.json", orient="records")

In [161]:
summary.to_json("../../data/data.json", orient="records")
cdData.to_json("../../data/cityAverage.json", orient="records")